# Trabajo Practico Nº1 - Parte 3: Comunicación y Sincronismo

## Información General

* **Universidad Nacional de la Matanza**
* Materia: Programación Concurrente
* Cuatrimestre: Segundo Cuatrimestre, Año 2024
* Profesores:
    * ADAGIO, MATIAS EZEQUIEL
    * CARNUCCIO, ESTEBAN
    * HIRSCHFELDT, DARIO
    * PALOMO, MAXIMO FACUNDO
    * VOLKER MARIANO LEONARDO
* Grupo: M4


* Integrantes:
    * ANTONIOLI, IVÁN OSCAR
    * DI NICCO, LUIS DEMETRIO
    * SANDOVAL VASQUEZ, JUAN LEANDRO
    * TIGANI MARTIN SEBASTIAN
    * VILLCA, LUIS ALBERTO

## Consigna

**Fecha de entrega**: 25/09/2024

**Forma de entrega**:
Se debe generar un informe que contenga los siguientes puntos:

*   **Carátula**: Con los integrantes del grupo.
*   **Link a un repositorio de GitHub**: En donde se encuentre el archivo del cuaderno de Colab generado para este Trabajo Práctico. Este archivo debe ser con la extension ipynb. También se debe subir a GitHub el código fuente.
*   **Conclusiones**: En esta sección se debe describir las dificultadas que encontraron al realizar el trabajo práctico.

Entregar el informe por plataforma de MIeL. Este debe ser en formato .pdf, con nombre TP1_Parte3_NumeroDelGrupo.pdf.

**Enunciado**: Una empresa desea optimizar el espacio utilizado en el edificio donde opera, para esto planean tener un solo baño con capacidad para **tres (3) empleados**, pero dado que actualmente su plantilla de empleados se compone de hombres y mujeres, la empresa quiere garantizar que en el baño no se encuentren hombres y mujeres al mismo tiempo.

Desarrolle un programa que implemente el funcionamiento de dicho baño utilizando algún método de comunicación y/o sincronización.

## Criterios a tener en cuenta

* [Coding Standard](https://google.github.io/styleguide/)
* Ajustes específicos (Ej. llaves en la misma columna)
* Espacios en lugar de tabulaciones (2 espacios)
* Métodos/funciones con menos de 15 líneas.
* Patrones de diseño
* No debe haber números mágicos, uso de constantes descriptivas.

## Resolución

### Código

In [ ]:
%%writefile accesoBaño.cpp
#include <iostream>
#include <mutex>
#include <semaphore>
#include <sstream>
#include <string>
#include <thread>
#include <unistd.h>
#include <vector>

using namespace std;
using namespace chrono;
using namespace this_thread;

mutex imprimirPantalla;
mutex accesoRegionCritica;
counting_semaphore<3> turno(3);

enum Baño
{
    VACIO = 0,
    HAY_HOMBRES = 1,
    HAY_MUJERES = 2
};

int cantidadMujeres = 0;
int cantidadHombres = 0;
int situacionBaño = VACIO;

void mujeres(int numero)
{
    string nombre = "Mujer " + to_string(numero);

    turno.acquire();
    if (situacionBaño == HAY_HOMBRES)
    {
        turno.release();
        while (situacionBaño == HAY_HOMBRES)
        {
            imprimirPantalla.lock();
            cout << nombre + " esperando" << endl;
            cout << "-----------------------------------------" << endl;
            imprimirPantalla.unlock();
            sleep_for(seconds(1));
        }
        turno.acquire();
    }
    accesoRegionCritica.lock();
    cantidadMujeres++;
    if (cantidadMujeres == 1)
    {
        situacionBaño = HAY_MUJERES;
    }
    imprimirPantalla.lock();
    cout << nombre + " entro al baño." << endl;
    cout << "\t- CantidadHombres: " + to_string(cantidadHombres) + "\n\t- CantidadMujeres: " + to_string(cantidadMujeres) << endl;
    cout << "-----------------------------------------" << endl;
    imprimirPantalla.unlock();
    accesoRegionCritica.unlock();

    sleep_for(seconds(2));

    accesoRegionCritica.lock();
    cantidadMujeres--;
    if (cantidadMujeres == 0)
    {
        situacionBaño = VACIO;
    }
    accesoRegionCritica.unlock();
    turno.release();
    imprimirPantalla.lock();
    cout << nombre + " sale del baño." << endl;
    cout << "-----------------------------------------" << endl;
    imprimirPantalla.unlock();
    sleep(0.1);
}

void hombres(int numero)
{
    string nombre = "Hombre " + to_string(numero);

    turno.acquire();
    if (situacionBaño == HAY_MUJERES)
    {
        turno.release();
        while (situacionBaño == HAY_MUJERES)
        {
            imprimirPantalla.lock();
            cout << nombre + " esperando" << endl;
            cout << "-----------------------------------------" << endl;
            imprimirPantalla.unlock();
        }
        turno.acquire();
    }
    accesoRegionCritica.lock();
    cantidadHombres++;
    if (cantidadHombres == 1)
    {
        situacionBaño = HAY_HOMBRES;
    }
    imprimirPantalla.lock();
    cout << nombre + " entro al baño." << endl;
    cout << "\t- CantidadHombres: " + to_string(cantidadHombres) + "\n\t- CantidadMujeres: " + to_string(cantidadMujeres)<< endl;
    cout << "-----------------------------------------" << endl;
    imprimirPantalla.unlock();
    accesoRegionCritica.unlock();

    sleep_for(seconds(2));

    accesoRegionCritica.lock();
    cantidadHombres--;
    if (cantidadHombres == 0)
    {
        situacionBaño = VACIO;
    }
    accesoRegionCritica.unlock();
    turno.release();
    imprimirPantalla.lock();
    cout << nombre + " sale del baño." << endl;
    cout << "-----------------------------------------" << endl;
    imprimirPantalla.unlock();
}

void agregarHilo(vector<thread> &grupo, int cantidadPersonas, void (*funcionGenero)(int))
{
    for (int i = 0; i < cantidadPersonas; i++)
    {
        grupo.push_back(thread(funcionGenero, i));
    }
}

void unirHilos(vector<thread> &grupo)
{
    for (auto &hilo : grupo)
    {
        if (hilo.joinable())
        {
            hilo.join();
        }
    }
}

int main(int argc, char *argv[])
{
    if (argc < 2)
    {
        cout << "Uso: " << argv[0] << " <cantidad de hombres y mujeres>" << endl;
        return EXIT_FAILURE;
    }

    int cantidadPersonas = stoi(argv[1]);

    if (cantidadPersonas < 0)
    {
        cout << "Ingrese cantidad de hombres y mujeres mayor a 0 " << endl;
        return EXIT_FAILURE;
    }
    cout << "-----------------------------------------" << endl;
    cout << "Comienzo de jornada laboral." << endl;
    cout << "-----------------------------------------" << endl;

    vector<thread> grupoHombres;

    vector<thread> grupoMujeres;

    agregarHilo(grupoHombres, cantidadPersonas, hombres);

    agregarHilo(grupoMujeres, cantidadPersonas, mujeres);

    unirHilos(grupoHombres);

    unirHilos(grupoMujeres);

    cout << "Fin de la jornada laboral." << endl;
    cout << "-----------------------------------------\n" << endl;

    return EXIT_SUCCESS;
}


Writing accesoBaño.cpp


### Compilación

Se le debe indicar al compilador que utilice el estándar C++20, que es una versión específica del lenguaje. En particular, el tipo `counting_semaphore` es una adición reciente introducida, por lo que se debe compilar con el estándar C++20 o posterior.

In [ ]:
!g++ -std=c++20 -o accesoBaño ./accesoBaño.cpp

### Ejecución


In [ ]:
!./accesoBaño 6

-----------------------------------------
Comienzo de jornada laboral.
-----------------------------------------
Hombre 3 entro al baño.
	- CantidadHombres: 1
	- CantidadMujeres: 0
-----------------------------------------
Hombre 2 entro al baño.
	- CantidadHombres: 2
	- CantidadMujeres: 0
-----------------------------------------
Hombre 1 entro al baño.
	- CantidadHombres: 3
	- CantidadMujeres: 0
-----------------------------------------
Hombre 3 sale del baño.
-----------------------------------------
Hombre 2 sale del baño.
-----------------------------------------
Hombre 4 entro al baño.
	- CantidadHombres: 2
	- CantidadMujeres: 0
-----------------------------------------
Hombre 5 entro al baño.
	- CantidadHombres: 3
	- CantidadMujeres: 0
-----------------------------------------
Hombre 1 sale del baño.
-----------------------------------------
Mujer 0 esperando
-----------------------------------------
Mujer 2 esperando
-----------------------------------------
Hombre 0 entro al b